In [1]:
import sys
sys.path.append("/p300s/wangmx_group/xutingfeng/SIS/")


In [2]:

from sis.dataset import SISDataset
import torch 
import transformers 
from sis.model.sismodel import DoubleTransformerModel
from sis.utils import modelParametersNum, try_gpu

# Step1 dataset load 
# device = try_gpu()
device = "cuda:1"

sisdataset = SISDataset(root_dir="/p300s/wangmx_group/xutingfeng/SIS/sis/dataset/total_data.csv", device = device)

aa_vocab = sisdataset.aa_vocab
sis_datasetDict = sisdataset.dataset_dict

train_dataloader = torch.utils.data.DataLoader(sis_datasetDict["train"], batch_size=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(sis_datasetDict["test"], batch_size=4, shuffle=True)




Using custom data configuration default-52200760334198e7
Reusing dataset csv (/home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-d7b3eee632703ea9.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-bec2fd355b0cb7d7.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0f1986b4010acd7d.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-fbd879c251d0f77b.arrow
Loading cached split indices for dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-f445a5304f7ba125.arrow and /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-cfda44d28db5f2dc.arrow


In [39]:
import numpy as np 
aa_embedding_dict = {aa:[1, 2, 3] for aa in aa_vocab.get_itos()}

aa_token_embedding_dict = {aa_vocab.lookup_indices([aa])[0]:value for aa, value in aa_embedding_dict.items()}

sorted_embeeding_tuple = sorted(aa_token_embedding_dict.items(), key = lambda x: x[0]) # (tokens, values) sorted by token at ascending order
sorted_embedding_array = np.stack([value for token, value in sorted_embeeding_tuple]).astype(np.float32)
sorted_embedding_array

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]], dtype=float32)

In [28]:

input_ = F.one_hot(torch.tensor([1, 0]), 22)
input_.shape

torch.Size([2, 22])

In [29]:
input_@sorted_embedding_array

tensor([[0.5484, 0.0738, 0.7047, 0.3437, 0.3530],
        [0.4896, 0.3018, 0.5982, 0.7547, 0.9737]], dtype=torch.float64)

In [32]:
input_

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [33]:
len(aa_vocab)

22

In [47]:
from torch import nn 
import numpy as np 
import torch.nn.functional as F 


class EmbeddingLayer(nn.Module):
    """
    EmbeddingLayer 传入aa_embeding_dict和aa_vocab
    aa_embedding_dict is {"A":[1, 2, 3], ...}
    aa_vocab is `torchtext.vocab.vocab.Vocab`

    这两个包含的token应该是一致的

    Example:
        from sis.dataset.vocab import build_vocab_from_alphabet_dict
        from sis.dataset.constants import BASE_AMINO_ACIDS

        aa_vocab = build_vocab_from_alphabet_dict(BASE_AMINO_ACIDS)
        aa_embedding_dict = {aa:np.random.randint(0, 5, size=(5)) for aa in aa_vocab.get_itos()}

        EMLayer = EmbeddingLayer(aa_embedding_dict=aa_embedding_dict, aa_vocab=aa_vocab)

        EMLayer(torch.tensor([1, 3, 5, 1, 2, 4]))
    """
    def __init__(self, aa_embedding_dict, aa_vocab):
        super(EmbeddingLayer, self).__init__()

        if len(aa_embedding_dict.keys()) != len(aa_vocab):
            raise ValueError("aa_embedding_dict should contain all token in aa_vocab, check whether special tokens are in aa_embedding_dict!")
        
        aa_token_embedding_dict = {aa_vocab.lookup_indices([aa])[0]:value for aa, value in aa_embedding_dict.items()}
        
        sorted_embeeding_tuple = sorted(aa_token_embedding_dict.items(), key = lambda x: x[0]) # (tokens, values) sorted by token at ascending order
        sorted_embedding_array = np.stack([value for token, value in sorted_embeeding_tuple]).astype(np.float32)

        self.sorted_embeeding_tuple = sorted_embeeding_tuple
        self.w = torch.tensor(sorted_embedding_array)
        self.d_model = self.w.shape[1]
        self.tokens_num = self.w.shape[0]
    
    def forward(self, x):
        x_onehot = F.one_hot(x, self.tokens_num).float()
        return x_onehot @ self.w 

        

In [52]:
from sis.dataset.vocab import build_vocab_from_alphabet_dict
from sis.dataset.constants import BASE_AMINO_ACIDS

aa_vocab = build_vocab_from_alphabet_dict(BASE_AMINO_ACIDS)
aa_embedding_dict = {aa:np.random.randint(0, 5, size=(5)) for aa in aa_vocab.get_itos()}

aa_embedding_dict

{'<unk>': array([2, 2, 1, 4, 1]),
 '<pad>': array([3, 0, 1, 3, 4]),
 'A': array([0, 2, 2, 3, 0]),
 'C': array([3, 1, 0, 4, 2]),
 'D': array([3, 2, 2, 3, 0]),
 'E': array([3, 3, 3, 0, 0]),
 'F': array([4, 4, 1, 3, 1]),
 'G': array([3, 1, 2, 1, 2]),
 'H': array([2, 0, 1, 2, 4]),
 'I': array([3, 3, 0, 4, 4]),
 'K': array([1, 0, 4, 3, 2]),
 'L': array([3, 4, 3, 4, 4]),
 'M': array([0, 2, 3, 4, 2]),
 'N': array([2, 2, 1, 2, 4]),
 'P': array([1, 2, 2, 0, 0]),
 'Q': array([4, 1, 0, 3, 4]),
 'R': array([4, 3, 1, 2, 2]),
 'S': array([2, 1, 4, 4, 2]),
 'T': array([4, 1, 2, 4, 4]),
 'V': array([3, 1, 2, 0, 0]),
 'W': array([2, 4, 1, 3, 1]),
 'Y': array([1, 3, 0, 2, 0])}

In [53]:
EMLayer = EmbeddingLayer(aa_embedding_dict=aa_embedding_dict, aa_vocab=aa_vocab)
EMLayer

EmbeddingLayer()

In [54]:

EMLayer(torch.tensor([1, 3, 5, 1, 2, 4]))

tensor([[3., 0., 1., 3., 4.],
        [3., 1., 0., 4., 2.],
        [3., 3., 3., 0., 0.],
        [3., 0., 1., 3., 4.],
        [0., 2., 2., 3., 0.],
        [3., 2., 2., 3., 0.]])

In [55]:
type(aa_vocab)

torchtext.vocab.vocab.Vocab

In [22]:
path="/p300s/wangmx_group/xutingfeng/SIS/sis/dataset/EmbeddingData/amino_acid_properties.csv"

expasy = pd.read_csv(path, sep=",",index_col=0).T
expasy

,pka_cooh_alpha,pka_nh3,pka_rgroup,isoelectric_points,molecularweight,numbercodons,bulkiness,polarityzimmerman,polaritygrantham,refractivity,...,coilroux,alpha_helixlevitt,beta_sheetlevitt,beta_turnlevitt,totalbeta_strand,antiparallelbeta_strand,parallelbeta_strand,a_a_composition,a_a_swiss_prot,relativemutability
ALA,2.35,9.87,7.00,6.11,89.0,4.0,11.50,0.00,8.1,4.34,...,0.824,1.29,0.90,0.77,0.92,0.90,1.00,8.3,8.25,100.0
ARG,2.18,9.09,13.20,10.76,174.0,6.0,14.28,52.00,10.5,26.66,...,0.893,0.96,0.99,0.88,0.93,1.02,0.68,5.7,5.53,65.0
ASN,2.18,9.09,13.20,10.76,132.0,2.0,12.82,3.38,11.6,13.28,...,1.167,0.90,0.76,1.28,0.60,0.62,0.54,4.4,4.06,134.0
ASP,1.88,9.60,3.65,2.98,133.0,2.0,11.68,49.70,13.0,12.00,...,1.197,1.04,0.72,1.41,0.48,0.47,0.50,5.3,5.45,106.0
CYS,1.71,10.78,8.33,5.02,121.0,1.0,13.46,1.48,5.5,35.77,...,0.953,1.11,0.74,0.81,1.16,1.24,0.91,1.7,1.37,20.0
GLN,2.17,9.13,7.00,5.65,146.0,2.0,14.45,3.53,10.5,17.56,...,0.947,1.27,0.80,0.98,0.95,1.18,0.28,4.0,3.93,93.0
GLU,2.19,9.67,4.25,3.08,147.0,2.0,13.57,49.90,12.3,17.26,...,0.761,1.44,0.75,0.99,0.61,0.62,0.59,6.2,6.75,102.0
GLY,2.34,9.60,7.00,6.06,75.0,4.0,3.40,0.00,9.0,0.00,...,1.251,0.56,0.92,1.64,0.61,0.56,0.79,7.2,7.07,49.0
HIS,1.78,8.97,5.97,7.64,155.0,2.0,13.69,51.60,10.4,21.81,...,1.068,1.22,1.08,0.68,0.93,1.12,0.38,2.2,2.27,66.0
ILE,2.32,9.76,7.00,6.04,131.0,3.0,21.40,0.13,5.2,19.06,...,0.886,0.97,1.45,0.51,1.81,1.54,2.60,5.2,5.96,96.0


In [2]:
from sis.dataset.EmbeddingData.parse import load_expasy_embedding_dict, load_meiler_embedding_dict

from sis.dataset.vocab import build_vocab_from_alphabet_dict
from sis.dataset.constants import BASE_AMINO_ACIDS
from sis.model.embedding import EmbeddingLayer

aa_embedding_dict = load_expasy_embedding_dict()
aa_vocab = build_vocab_from_alphabet_dict(BASE_AMINO_ACIDS)

EMLayer = EmbeddingLayer(aa_embedding_dict=aa_embedding_dict, aa_vocab=aa_vocab)
